In [1]:
from transformers import pipeline

In [2]:

classifier = pipeline(
    "zero-shot-classification",
    model="FacebookAI/roberta-large-mnli",
    hypothesis_template="Under which topic would I see the following article in the news? The topics are: {}.",
    batch_size=64
)

Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [7]:
# titles = ["OpenAI releases GPT‑5", "Global markets rally on tech earnings", "Israel appoints new defense minister", "World Cup final ends in dramatic fashion", "New AI regulations proposed by EU", "OpenAI releases GPT‑5", "Global markets rally on tech earnings", "Israel appoints new defense minister", "World Cup final ends in dramatic fashion", "New AI regulations proposed by EU", ] * 5
titles = ["OpenAI releases GPT‑5", "Global markets rally on tech earnings"] * 32
topics = ["Technology", "Finance", "Sports", "Health", "Politics"]

results = classifier(titles, candidate_labels=topics)

In [6]:
results

[{'sequence': 'OpenAI releases GPT‑5',
  'labels': ['Technology', 'Politics', 'Health', 'Sports', 'Finance'],
  'scores': [0.3227241337299347,
   0.2182881385087967,
   0.16146892309188843,
   0.1549658477306366,
   0.14255298674106598]},
 {'sequence': 'Global markets rally on tech earnings',
  'labels': ['Technology', 'Finance', 'Politics', 'Health', 'Sports'],
  'scores': [0.5121206045150757,
   0.14426761865615845,
   0.1273437887430191,
   0.1105947494506836,
   0.10567331314086914]},
 {'sequence': 'OpenAI releases GPT‑5',
  'labels': ['Technology', 'Politics', 'Health', 'Sports', 'Finance'],
  'scores': [0.3227241337299347,
   0.2182881385087967,
   0.16146892309188843,
   0.1549658477306366,
   0.14255298674106598]},
 {'sequence': 'Global markets rally on tech earnings',
  'labels': ['Technology', 'Finance', 'Politics', 'Health', 'Sports'],
  'scores': [0.5121206045150757,
   0.14426761865615845,
   0.1273437887430191,
   0.1105947494506836,
   0.10567331314086914]},
 {'sequence'